In [1]:
pip install pandas sqlalchemy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 12.7 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
#Loading required libraries
import pandas as pd
import sqlite3
from sqlite3 import Error
import os
from sqlalchemy import create_engine



### Loading dataset

In [2]:
# Set the directory path where the CSV files are located
# folder_path = "/Users/lawshia/Desktop/DMQL Project/Data/dunnhumby_Let's-Get-Sort-of-Real-(Sample-5K-customers)"
folder_path = "/Users/ayushkaurav/Downloads/Customer-Behavior-Analytics-Database-main/Datafile"

# Get the list of files in the directory
file_list = os.listdir(folder_path)

# Initialize an empty list to store DataFrames from each file
dataframes = []

# Loop through each file in the list
for file_name in file_list:
    # Check if the file is in CSV format
    if file_name.endswith(".csv"):
        # Construct the full file path
        file_path = os.path.join(folder_path, file_name)
        
        # Read the CSV file into a DataFrame
        df = pd.read_csv(file_path)
       
        if 'SHOP_WEEK' in df.columns:
            sampled_df = df.sample(n=10000, random_state=42)
            dataframes.append(sampled_df)
            # Concatenate all DataFrames into a single DataFrame
            combined_df = pd.concat(dataframes, ignore_index=True)

combined_df['RowNumber'] = range(1, len(combined_df) + 1)

combined_df['RowNumber'].nunique()            
            
# # Print the combined DataFrame
print(f'Shape:{combined_df.shape}')



#Cleaning to drop rows with missing customer code
combined_df = combined_df[~combined_df['CUST_CODE'].isnull()]
combined_df.head(10)

Shape:(460000, 23)


,SHOP_WEEK,SHOP_DATE,SHOP_WEEKDAY,SHOP_HOUR,QUANTITY,SPEND,PROD_CODE,PROD_CODE_10,PROD_CODE_20,PROD_CODE_30,...,CUST_LIFESTAGE,BASKET_ID,BASKET_SIZE,BASKET_PRICE_SENSITIVITY,BASKET_TYPE,BASKET_DOMINANT_MISSION,STORE_CODE,STORE_FORMAT,STORE_REGION,RowNumber
0,200648,20070128,1,18,1,2.03,PRD0901825,CL00129,DEP00046,G00013,...,OT,994104200334249,M,LA,Top Up,Mixed,STORE00703,LS,N03,1
1,200648,20070124,4,21,1,0.64,PRD0900801,CL00110,DEP00039,G00010,...,OT,994104200303399,L,UM,Top Up,Grocery,STORE00996,LS,N02,2
2,200648,20070127,7,17,1,28.82,PRD0904300,CL00058,DEP00016,G00006,...,OT,994104200219916,L,UM,Full Shop,Fresh,STORE02450,LS,N02,3
3,200648,20070126,6,14,1,2.93,PRD0900123,CL00148,DEP00052,G00015,...,YF,994104200253687,L,LA,Top Up,Mixed,STORE01113,LS,W02,4
4,200648,20070128,1,21,1,1.00,PRD0902029,CL00031,DEP00008,G00004,...,YF,994104200335936,L,MM,Top Up,Fresh,STORE02653,LS,W02,5
5,200648,20070123,3,16,1,1.25,PRD0901283,CL00152,DEP00052,G00015,...,OT,994104200477329,L,MM,Top Up,Grocery,STORE01969,LS,W02,6
6,200648,20070122,2,19,1,0.96,PRD0904836,CL00043,DEP00011,G00004,...,OF,994104200318027,L,UM,Top Up,Mixed,STORE01383,LS,S01,7
7,200648,20070126,6,20,1,0.61,PRD0903977,CL00179,DEP00061,G00018,...,OT,994104200713295,L,MM,Full Shop,Fresh,STORE02283,LS,W01,8
8,200648,20070127,7,21,1,0.89,PRD0903228,CL00138,DEP00048,G00013,...,YF,994104200474466,L,LA,Full Shop,Mixed,STORE01490,LS,E01,9
9,200648,20070128,1,18,1,1.82,PRD0902749,CL00126,DEP00046,G00013,...,OT,994104200300723,L,MM,Full Shop,Grocery,STORE00201,MS,W02,10


In [3]:
combined_df.columns

Index(['SHOP_WEEK', 'SHOP_DATE', 'SHOP_WEEKDAY', 'SHOP_HOUR', 'QUANTITY',
       'SPEND', 'PROD_CODE', 'PROD_CODE_10', 'PROD_CODE_20', 'PROD_CODE_30',
       'PROD_CODE_40', 'CUST_CODE', 'CUST_PRICE_SENSITIVITY', 'CUST_LIFESTAGE',
       'BASKET_ID', 'BASKET_SIZE', 'BASKET_PRICE_SENSITIVITY', 'BASKET_TYPE',
       'BASKET_DOMINANT_MISSION', 'STORE_CODE', 'STORE_FORMAT', 'STORE_REGION',
       'RowNumber'],
      dtype='object')

In [4]:
def create_connection(db_file, delete_db=False):
    '''
        Purpose:
            To create database connection to retrieve tables
        Input: 
            db_file - name of the db file

        Output:
            conn - This establishes connection to the database

    '''    

    import os
    if delete_db and os.path.exists(db_file):
        os.remove(db_file)

    conn = None
    try:
        conn = sqlite3.connect(db_file)
        conn.execute("PRAGMA foreign_keys = 1")
    except Error as e:
        print(e)

    return conn


def create_table(conn, create_table_sql):
    '''
        Input: 
            conn - database on which the table will be created
            create_table_sql - Input create table query

        Output:
            e - error if any issue comes up

    '''
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)
        
def execute_sql_statement(sql_statement, conn):
    '''
        Input: 
            sql_statement - SQL statement to be executed
            conn - database on which the table will be created

        Output:
            rows - this returns the rows that's being created

    '''
    cur = conn.cursor()
    cur.execute(sql_statement)

    rows = cur.fetchall()

    return rows



In [7]:
# Create database connections
conn_normalized = create_connection('normalized.db')
conn_source=create_connection('my_database.db')

db_url = 'sqlite:///my_database.db'
engine = create_engine(db_url)



In [8]:
# converting dataframe to database
combined_df.to_sql('all_transactions', engine, if_exists='replace', index=False)


369610

#-------------------------------------------------------------------------------------------------------------#

### Product Code 40 Table

In [9]:
drop_table_sql = """
        DROP TABLE IF EXISTS product_40_code;
    """ 
execute_sql_statement(drop_table_sql, conn_normalized)

create_table_sql = """
    CREATE TABLE [product_40_code] 
    (
    [PROD_CODE_30] TEXT NOT NULL PRIMARY KEY,
    [PROD_CODE_40] TEXT NOT NULL
    );
"""

sql_statement = "SELECT distinct PROD_CODE_30, PROD_CODE_40 FROM all_transactions"
create_table(conn_normalized, create_table_sql)
degrees = execute_sql_statement(sql_statement, conn_source)

def insert_degree(conn, values):
    sql = ''' INSERT INTO product_40_code (PROD_CODE_30, PROD_CODE_40)
              VALUES(?,?) '''
    cur = conn.cursor()
    cur.execute(sql, values)
    return cur.lastrowid


with conn_normalized:
    for degree in degrees:
        insert_degree(conn_normalized, degree)
        
print("\033[1mSample Dataset\033[0m")  

sql_statement = '''
SELECT 
distinct
*
from 
product_40_code
'''

df = pd.read_sql_query(sql_statement, conn_normalized)
df.head()

Sample Dataset


,PROD_CODE_30,PROD_CODE_40
0,G00013,D00003
1,G00010,D00003
2,G00006,D00002
3,G00015,D00003
4,G00004,D00002


### Relation Schema
#### product_40_code ('PROD_CODE_30', 'PROD_CODE_40')

#-------------------------------------------------------------------------------------------------------------#

### Product Code 30 Table

In [10]:
drop_table_sql = """
        DROP TABLE IF EXISTS product_30_code;
    """ 
execute_sql_statement(drop_table_sql, conn_normalized)

create_table_sql = """
    CREATE TABLE [product_30_code] 
    (
    [PROD_CODE_20] TEXT NOT NULL PRIMARY KEY,
    [PROD_CODE_30] TEXT NOT NULL
    );
"""

sql_statement = "SELECT distinct PROD_CODE_20, PROD_CODE_30 FROM all_transactions"
create_table(conn_normalized, create_table_sql)
degrees = execute_sql_statement(sql_statement, conn_source)

def insert_degree(conn, values):
    sql = ''' INSERT INTO product_30_code (PROD_CODE_20, PROD_CODE_30)
              VALUES(?,?) '''
    cur = conn.cursor()
    cur.execute(sql, values)
    return cur.lastrowid


with conn_normalized:
    for degree in degrees:
        insert_degree(conn_normalized, degree)
        
print("\033[1mSample Dataset\033[0m")

sql_statement = '''
SELECT 
distinct
*
from 
product_30_code
'''

df = pd.read_sql_query(sql_statement, conn_normalized)
df.head()


Sample Dataset


,PROD_CODE_20,PROD_CODE_30
0,DEP00046,G00013
1,DEP00039,G00010
2,DEP00016,G00006
3,DEP00052,G00015
4,DEP00008,G00004


### Relation Schema
#### product_30_code ('PROD_CODE_10', 'PROD_CODE_30')

#-------------------------------------------------------------------------------------------------------------#

### Product Code 20 Table

In [11]:
drop_table_sql = """
        DROP TABLE IF EXISTS product_20_code;
    """ 
execute_sql_statement(drop_table_sql, conn_normalized)

create_table_sql = """
    CREATE TABLE [product_20_code] 
    (
    [PROD_CODE_10] TEXT NOT NULL PRIMARY KEY,
    [PROD_CODE_20] TEXT NOT NULL
    );
"""

sql_statement = "SELECT distinct PROD_CODE_10, PROD_CODE_20 FROM all_transactions"
create_table(conn_normalized, create_table_sql)
degrees = execute_sql_statement(sql_statement, conn_source)

def insert_degree(conn, values):
    sql = ''' INSERT INTO product_20_code (PROD_CODE_10, PROD_CODE_20)
              VALUES(?,?) '''
    cur = conn.cursor()
    cur.execute(sql, values)
    return cur.lastrowid


with conn_normalized:
    for degree in degrees:
        insert_degree(conn_normalized, degree)

        
print("\033[1mSample Dataset\033[0m")

sql_statement = '''
SELECT 
distinct
*
from 
product_20_code
'''

df = pd.read_sql_query(sql_statement, conn_normalized)
df.head()


Sample Dataset


,PROD_CODE_10,PROD_CODE_20
0,CL00129,DEP00046
1,CL00110,DEP00039
2,CL00058,DEP00016
3,CL00148,DEP00052
4,CL00031,DEP00008


### Relation Schema
#### product_20_code ('PROD_CODE_10', 'PROD_CODE_20')

#-------------------------------------------------------------------------------------------------------------#

### Product Code 10 Table

In [12]:


drop_table_sql = """
        DROP TABLE IF EXISTS product_10_code;
    """ 
execute_sql_statement(drop_table_sql, conn_normalized)

create_table_sql = """
    CREATE TABLE [product_10_code] 
    (
    [PROD_CODE] TEXT NOT NULL PRIMARY KEY,
    [PROD_CODE_10] TEXT NOT NULL
    );
"""

sql_statement = "SELECT distinct PROD_CODE, PROD_CODE_10 FROM all_transactions"
create_table(conn_normalized, create_table_sql)
degrees = execute_sql_statement(sql_statement, conn_source)

def insert_degree(conn, values):
    sql = ''' INSERT INTO product_10_code (PROD_CODE, PROD_CODE_10)
              VALUES(?,?) '''
    cur = conn.cursor()
    cur.execute(sql, values)
    return cur.lastrowid


with conn_normalized:
    for degree in degrees:
        insert_degree(conn_normalized, degree)
       
    
print("\033[1mSample Dataset\033[0m")

sql_statement = '''
SELECT 
distinct
*
from 
product_10_code
'''

df = pd.read_sql_query(sql_statement, conn_normalized)
df.head()
        

Sample Dataset


,PROD_CODE,PROD_CODE_10
0,PRD0901825,CL00129
1,PRD0900801,CL00110
2,PRD0904300,CL00058
3,PRD0900123,CL00148
4,PRD0902029,CL00031


### Relation Schema
#### product_10_code ('PROD_CODE', 'PROD_CODE_10')


#-------------------------------------------------------------------------------------------------------------#

### Customer Detail Tables

In [13]:


drop_table_sql = """
        DROP TABLE IF EXISTS Customer_details;
    """ 
execute_sql_statement(drop_table_sql, conn_normalized)

create_table_sql = """
    CREATE TABLE [Customer_details] 
    (
    [CUST_CODE] TEXT NOT NULL PRIMARY KEY,
    [CUST_PRICE_SENSITIVITY] TEXT NOT NULL,
    [CUST_LIFESTAGE] TEXT 
    );
"""

sql_statement = "SELECT distinct CUST_CODE, CUST_PRICE_SENSITIVITY, CUST_LIFESTAGE FROM all_transactions"
create_table(conn_normalized, create_table_sql)
degrees = execute_sql_statement(sql_statement, conn_source)

def insert_degree(conn, values):
    sql = ''' INSERT INTO Customer_details (CUST_CODE, CUST_PRICE_SENSITIVITY, CUST_LIFESTAGE)
              VALUES(?,?,?) '''
    cur = conn.cursor()
    cur.execute(sql, values)
    return cur.lastrowid


with conn_normalized:
    for degree in degrees:
        insert_degree(conn_normalized, degree)
        
print("\033[1mSample Dataset\033[0m")

sql_statement = '''
SELECT 
distinct
*
from 
Customer_details
'''

df = pd.read_sql_query(sql_statement, conn_normalized)
df.head()
        

Sample Dataset


,CUST_CODE,CUST_PRICE_SENSITIVITY,CUST_LIFESTAGE
0,CUST0000277824,MM,OT
1,CUST0000228256,UM,OT
2,CUST0000095739,UM,OT
3,CUST0000149372,LA,YF
4,CUST0000280589,UM,YF


### Relation Schema
#### Customer_details ('CUST_CODE', 'CUST_PRICE_SENSITIVITY', 'CUST_LIFESTAGE')


#-------------------------------------------------------------------------------------------------------------#

### Basket Details Table 

In [14]:
drop_table_sql = """
        DROP TABLE IF EXISTS basket_details;
    """ 
execute_sql_statement(drop_table_sql, conn_normalized)

create_table_sql = """
    CREATE TABLE [basket_details] 
    (
    [BASKET_ID] TEXT NOT NULL PRIMARY KEY,
    [BASKET_SIZE] TEXT NOT NULL,
    [BASKET_PRICE_SENSITIVITY] TEXT NOT NULL,
    [BASKET_TYPE] TEXT NOT NULL,
    [BASKET_DOMINANT_MISSION] TEXT NOT NULL
    );
"""

sql_statement = "SELECT distinct BASKET_ID,BASKET_SIZE, BASKET_PRICE_SENSITIVITY,BASKET_TYPE,BASKET_DOMINANT_MISSION FROM all_transactions"
create_table(conn_normalized, create_table_sql)
degrees = execute_sql_statement(sql_statement, conn_source)

def insert_degree(conn, values):
    sql = ''' INSERT INTO basket_details (BASKET_ID,BASKET_SIZE, BASKET_PRICE_SENSITIVITY,BASKET_TYPE,BASKET_DOMINANT_MISSION)
              VALUES(?,?,?,?,?) '''
    cur = conn.cursor()
    cur.execute(sql, values)
    return cur.lastrowid


with conn_normalized:
    for degree in degrees:
        insert_degree(conn_normalized, degree)
        
print("\033[1mSample Dataset\033[0m")       
        
sql_statement = '''
SELECT 
distinct
*
from 
basket_details
'''

df = pd.read_sql_query(sql_statement, conn_normalized)
df.head()

Sample Dataset


,BASKET_ID,BASKET_SIZE,BASKET_PRICE_SENSITIVITY,BASKET_TYPE,BASKET_DOMINANT_MISSION
0,994104200334249,M,LA,Top Up,Mixed
1,994104200303399,L,UM,Top Up,Grocery
2,994104200219916,L,UM,Full Shop,Fresh
3,994104200253687,L,LA,Top Up,Mixed
4,994104200335936,L,MM,Top Up,Fresh


### Relation Schema
#### basket_details ('BASKET_ID', 'BASKET_SIZE', 'BASKET_PRICE_SENSITIVITY', 'BASKET_TYPE','BASKET_DOMINANT_MISSION')


#-------------------------------------------------------------------------------------------------------------#

### Store Details Table

In [15]:


drop_table_sql = """
        DROP TABLE IF EXISTS Store_details;
    """ 
execute_sql_statement(drop_table_sql, conn_normalized)

create_table_sql = """
    CREATE TABLE [Store_details] 
    (
    [STORE_CODE] TEXT NOT NULL PRIMARY KEY,
    [STORE_FORMAT] TEXT NOT NULL,
    [STORE_REGION] TEXT NOT NULL
    );
"""

sql_statement = "SELECT distinct STORE_CODE,STORE_FORMAT,STORE_REGION FROM all_transactions"
create_table(conn_normalized, create_table_sql)
degrees = execute_sql_statement(sql_statement, conn_source)

def insert_degree(conn, values):
    sql = ''' INSERT INTO Store_details (STORE_CODE,STORE_FORMAT,STORE_REGION)
              VALUES(?,?,?) '''
    cur = conn.cursor()
    cur.execute(sql, values)
    return cur.lastrowid


with conn_normalized:
    for degree in degrees:
        insert_degree(conn_normalized, degree)
        
print("\033[1mSample Dataset\033[0m")

sql_statement = '''
SELECT 
distinct
*
from 
Store_details
'''

df = pd.read_sql_query(sql_statement, conn_normalized)
df.head()

Sample Dataset


,STORE_CODE,STORE_FORMAT,STORE_REGION
0,STORE00703,LS,N03
1,STORE00996,LS,N02
2,STORE02450,LS,N02
3,STORE01113,LS,W02
4,STORE02653,LS,W02


### Relation Schema
#### Store_details (''STORE_CODE', 'STORE_FORMAT', 'STORE_REGION'')


#-------------------------------------------------------------------------------------------------------------#

### Transaction details tables

In [16]:
drop_table_sql = """
        DROP TABLE IF EXISTS transaction_details;
    """ 
execute_sql_statement(drop_table_sql, conn_normalized)

create_table_sql = """
    CREATE TABLE [transaction_details] 
    (
    [RowNumber] INTEGER NOT NULL PRIMARY KEY,
    [SHOP_WEEK] TEXT NOT NULL,
    [SHOP_DATE] TEXT NOT NULL,
    [SHOP_WEEKDAY] TEXT NOT NULL,
    [SHOP_HOUR] TEXT NOT NULL,
    [QUANTITY] TEXT NOT NULL,
    [SPEND] TEXT NOT NULL,
    [PROD_CODE] TEXT NOT NULL,
    [CUST_CODE] TEXT NOT NULL,
    [BASKET_ID] TEXT NOT NULL,
    [STORE_CODE] TEXT NOT NULL,
    FOREIGN KEY(CUST_CODE) REFERENCES Customer_details(CUST_CODE)  ON DELETE CASCADE,
    FOREIGN KEY(STORE_CODE) REFERENCES Store_details(STORE_CODE)  ON DELETE CASCADE,
    FOREIGN KEY(PROD_CODE) REFERENCES product_10_code(PROD_CODE)  ON DELETE CASCADE
    );
"""

sql_statement = "SELECT distinct SHOP_WEEK, SHOP_DATE, SHOP_WEEKDAY, SHOP_HOUR, QUANTITY, SPEND, PROD_CODE, CUST_CODE, BASKET_ID, STORE_CODE FROM all_transactions"
create_table(conn_normalized, create_table_sql)
degrees = execute_sql_statement(sql_statement, conn_source)

def insert_degree(conn, values):
    sql = ''' INSERT INTO transaction_details (SHOP_WEEK, SHOP_DATE, SHOP_WEEKDAY, SHOP_HOUR, QUANTITY, SPEND, PROD_CODE, CUST_CODE, BASKET_ID, STORE_CODE)
              VALUES(?,?,?,?,?,?,?,?,?,?) '''
    cur = conn.cursor()
    cur.execute(sql, values)
    return cur.lastrowid


with conn_normalized:
    for degree in degrees:
        insert_degree(conn_normalized, degree)
        
print("\033[1mSample Dataset\033[0m")

sql_statement = '''
SELECT 
distinct
*
from 
transaction_details
'''

df = pd.read_sql_query(sql_statement, conn_normalized)
df.head()        

Sample Dataset


,RowNumber,SHOP_WEEK,SHOP_DATE,SHOP_WEEKDAY,SHOP_HOUR,QUANTITY,SPEND,PROD_CODE,CUST_CODE,BASKET_ID,STORE_CODE
0,1,200648,20070128,1,18,1,2.03,PRD0901825,CUST0000277824,994104200334249,STORE00703
1,2,200648,20070124,4,21,1,0.64,PRD0900801,CUST0000228256,994104200303399,STORE00996
2,3,200648,20070127,7,17,1,28.82,PRD0904300,CUST0000095739,994104200219916,STORE02450
3,4,200648,20070126,6,14,1,2.93,PRD0900123,CUST0000149372,994104200253687,STORE01113
4,5,200648,20070128,1,21,1,1.0,PRD0902029,CUST0000280589,994104200335936,STORE02653


### Relation Schema
#### transaction_details ('RowNumber', 'SHOP_WEEK', 'SHOP_DATE', 'SHOP_WEEKDAY', 'SHOP_HOUR','QUANTITY', 'SPEND', 'PROD_CODE', 'CUST_CODE', 'BASKET_ID','STORE_CODE')


In [17]:
#### Checking tables in the database
import sqlite3

# Connect to the SQLite database
connection = sqlite3.connect('normalized.db')

# Create a cursor
cursor = connection.cursor()

# Query the sqlite_master table to get table names
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

# Fetch and print the table names
tables = cursor.fetchall()
for table in tables:
    print(table[0])

# Close the cursor and connection
cursor.close()
connection.close()


product_40_code
product_30_code
product_20_code
product_10_code
Customer_details
basket_details
Store_details
transaction_details
